# Constructing baseline

In [1]:
%pip install torch torchvision
%pip install jupyter ipywidgets
%pip install "pytorch-lightning <1.9" lightning-flash[image]

Note: you may need to restart the kernel to use updated packages.
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.9/285.9 kB 4.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.8/529.8 kB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 kB 5.7 MB/s eta 0:00:00
  Using cached jupyterlab_pygments-0.2.2-py2.py3-none-any.whl (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 5.4 MB/s eta 0:00:00
  Using cached bleach-6.0.0-py3-none-any.whl (162 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.5 MB/s eta 0:00:00
  Using cached importlib_metadata-6.6.0-py3-none-any.whl (22 kB)
  Usi

In [2]:
from pytorch_lightning import seed_everything

import flash
from flash.core.classification import LabelsOutput
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier

from flash.core.data.transforms import ApplyToKeys
from flash.core.data.io.input_transform import InputTransform
from dataclasses import dataclass
from torch.utils.data import Subset
from torchvision import datasets, transforms
from random import shuffle
import torch
from tqdm.notebook import tqdm
from torchmetrics import Recall, F1Score

/DATA/souraviai/miniconda3/envs/tmp/lib/python3.8/site-packages/pl_bolts/callbacks/data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
/DATA/souraviai/miniconda3/envs/tmp/lib/python3.8/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/DATA/souraviai/mini

In [3]:
# set the random seeds.
seed_everything(42)

# 1. Download and organize the data
# download_data("https://pl-flash-data.s3.amazonaws.com/hymenoptera_data.zip", "data/")

ds = datasets.SUN397(
    root='./data', 
)
print(ds)

s = [[] for _ in ds.classes]
for i, label in tqdm(enumerate(ds._labels)):
    s[label].append(i)

idxs = {"train": [], "val": [], "test": []}
for idx_list in tqdm(s):
    shuffle(idx_list)
    idxs["val"].extend(idx_list[:10])
    idxs["test"].extend(idx_list[10:10 + 40])
    idxs["train"].extend(idx_list[50:])
# print(idxs)

Global seed set to 42


Dataset SUN397
    Number of datapoints: 108754
    Root location: ./data


0it [00:00, ?it/s]

  0%|          | 0/397 [00:00<?, ?it/s]

In [4]:
from typing import Tuple


@dataclass
class ImageClassificationInputTransform(InputTransform):
    image_size: Tuple[int, int] = (32, 32)
    # mean: Union[float, Tuple[float, float, float]] = (0.485, 0.456, 0.406)
    # std: Union[float, Tuple[float, float, float]] = (0.229, 0.224, 0.225)

    def per_sample_transform(self):
        return transforms.Compose(
            [
                ApplyToKeys(
                    "input",
                    transforms.Compose(
                        [
                            transforms.ToTensor(), 
                            # transforms.Normalize(self.mean, self.std)
                        ]
                    ),
                ),
                ApplyToKeys("target", torch.as_tensor),
            ]
        )

    def train_per_sample_transform(self):
        return transforms.Compose(
            [
                ApplyToKeys(
                    "input",
                    transforms.Compose(
                        [
                            transforms.RandomResizedCrop(self.image_size),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            # transforms.Normalize(self.mean, self.std),
                        ]
                    ),
                ),
                ApplyToKeys("target", torch.as_tensor),
            ]
        )

In [5]:
datamodule = ImageClassificationData.from_datasets(
    train_dataset=Subset(ds, idxs["train"]),
    val_dataset=Subset(ds, idxs["val"]),
    test_dataset=Subset(ds, idxs["test"]),
    batch_size=128,
    transform=ImageClassificationInputTransform(),
)
# print(datamodule.num_classes)

# 2. Build the model using desired Task
model = ImageClassifier(
    backbone="resnet18", 
    num_classes=len(s), 
    pretrained=False,
    # metrics=F1Score(task='multiclass', num_classes=len(s)),
)
print(model)
# 3. Create the trainer (run one epoch for demo)
trainer = flash.Trainer(max_epochs=1, accelerator="gpu", devices=torch.cuda.device_count())
print(trainer)
# 4. Train the model
trainer.fit(model, datamodule=datamodule)

# 5. Save the model!
trainer.save_checkpoint("test/image_classification_model.pt")

/DATA/souraviai/miniconda3/envs/tmp/lib/python3.8/site-packages/pytorch_lightning/utilities/apply_func.py:31: LightningDeprecationWarning: `pytorch_lightning.utilities.apply_func.apply_to_collection` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_utilities.core.apply_func.apply_to_collection` instead.
  rank_zero_deprecation(


ModuleNotFoundError: Required dependencies not available. Please run: pip install 'lightning-flash[image]'

In [1]:
%pip freeze | grep lightning

lightning-bolts==0.6.0.post1
lightning-flash==0.8.1.post0
lightning-utilities==0.8.0
pytorch-lightning==1.8.6
Note: you may need to restart the kernel to use updated packages.


# Testing

In [2]:
import torch 

In [3]:
torch.cuda.device_count()

2

In [ ]:
%conda install imbalanced-learn

In [ ]:
from imblearn.metrics import geometric_mean_score, sensitivity_score

In [ ]:
import numpy as np


y = np.random.randint(10, size=500)
pred = np.random.randint(10, size=500)
# print(y)
# print(pred)
tmp1 = sensitivity_score(y, pred, average=None)
print(tmp1)
tmp2 = geometric_mean_score(y, pred)
print(tmp2)

In [ ]:
acc = np.load("./logs/Imbalance_S320_M2m_L0.5_W160_E0.1_I10_svhn_R60_resnet32_G0.9_B0.999/classwise_acc.npy")
r =1 
for i in acc**(1/len(acc)):
    r *= i

print(r)

In [ ]:
acc.mean()

In [59]:
from torchvision import datasets, transforms
from utils import get_mean_and_std
from torch.utils.data import DataLoader

data = datasets.SUN397(
    root='./data', 
    # split='train',
    transform=transforms.Compose(
        [
            transforms.ToTensor(),

        ]
    )
)

# dataloader = DataLoader(
#     data, 
#     batch_size=1, 
#     # shuffle=True, 
#     num_workers=2
# )

# m, s = get_mean_and_std(data)
# print(m)
# print(s)
# data = data.data / 255 # data is numpy array

# print(np.round(data.mean(axis = (0,2,3)), 4))
# print(np.round(data.std(axis = (0,2,3)), 4))

In [2]:
from torchvision import datasets

In [ ]:
from fuzzywuzzy import process
print(process.extractOne('sun37'.lower(), datasets.__all__, lambda x: x.lower())[0])

SUN397


In [2]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)

Number of CPU cores: 24
